In [28]:
%pip install sshtunnel
%pip install requests
%pip install psycopg
%pip install pgvector
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
%load_ext dotenv
%dotenv

## Create Helper for the DB connection

In [3]:
from sshtunnel import SSHTunnelForwarder
import requests
import json
import os
import pgvector
import psycopg
from pgvector.psycopg import register_vector
import json
def get_db_connection():
    # Setting up the SSH tunnel with tunnel credentials
    REMOTE_HOST = os.getenv("REMOTE_HOST")
    REMOTE_SSH_PORT = int(os.getenv("REMOTE_SSH_PORT"))
    PORT = int(os.getenv("PORT"))
    SSH_KEYFILE = os.getenv("SSH_KEYFILE")
    SSH_USERNAME =  os.getenv("SSH_USERNAME")

    server = SSHTunnelForwarder(
        ssh_address_or_host=(REMOTE_HOST, REMOTE_SSH_PORT),
        ssh_username= SSH_USERNAME,
        ssh_pkey=SSH_KEYFILE,
        # Key part! Connect to AWS_HOST through the tunnel.
        remote_bind_address=('localhost', PORT)
    )
    server.start()
    print("server connected")

    conn_str = f"dbname=postgres host=localhost port={server.local_bind_port} user=postgres password={os.getenv('DB_PASSWORD')}"
    return psycopg.connect(conn_str)

## Define Helper Methods to scrape website to get data

In [4]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


## Get all the cases in a volume

In [10]:
import requests
from bs4 import BeautifulSoup

def get_links_in_volume(reporter,volume):
    # URL of the page to parse
    url = f'https://static.case.law/{reporter}/{volume}/cases/'

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all <a> tags with href attributes
        links = soup.find_all('a', href=True)
        links_array = []
        # Extract and print the file names
        for link in links:
            file_name = link['href'].split('/')[-1]
            #print(file_name)
            if not file_name == '':
                links_array.append(file_name)
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

    return links_array


In [9]:
get_links_in_volume(REPORTER, 1)

['0001-01.json',
 '0006-01.json',
 '0014-01.json',
 '0019-01.json',
 '0027-01.json',
 '0033-01.json',
 '0039-01.json',
 '0043-01.json',
 '0046-01.json',
 '0051-01.json',
 '0053-01.json',
 '0056-01.json',
 '0061-01.json',
 '0065-01.json',
 '0067-01.json',
 '0077-01.json',
 '0081-01.json',
 '0085-01.json',
 '0090-01.json',
 '0094-01.json',
 '0104-01.json',
 '0109-01.json',
 '0112-01.json',
 '0116-01.json',
 '0124-01.json',
 '0129-01.json',
 '0133-01.json',
 '0141-01.json',
 '0148-01.json',
 '0152-01.json',
 '0159-01.json',
 '0164-01.json',
 '0174-01.json',
 '0177-01.json',
 '0183-01.json',
 '0185-01.json',
 '0189-01.json',
 '0192-01.json',
 '0194-01.json',
 '0196-01.json',
 '0202-01.json',
 '0218-01.json',
 '0224-01.json',
 '0226-01.json',
 '0230-01.json',
 '0236-01.json',
 '0242-01.json',
 '0250-01.json',
 '0257-01.json',
 '0263-01.json',
 '0270-01.json',
 '0272-01.json',
 '0277-01.json',
 '0285-01.json',
 '0292-01.json',
 '0299-01.json',
 '0303-01.json',
 '0305-01.json',
 '0315-01.json

## Get the number of vomules in a reporter

In [6]:
def get_number_of_volumes_in_reporter(reporter_name):
        # URL of the page to parse
    url = f'https://static.case.law/{reporter_name}/'
    # Send a GET request to the URL
    response = requests.get(url)
    volume_array = []

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all <a> tags with href attributes
        links = soup.find_all('a', href=True)
        # Extract and print the file names
        for link in links:
            file_name = link['href'].split('/')[-1]
            #print(file_name)
            if not file_name.endswith('json'):
                volume_array.append(file_name)
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

    return volume_array[-1].split(".")[0]

In [7]:
get_number_of_volumes_in_reporter(REPORTER)

'199'

## Select Reporter

In [8]:
REPORTER = "wash-app"
NUM_OF_VOLUMES = int(get_number_of_volumes_in_reporter(REPORTER))

## Create Volumes Table

In [6]:

# Setting up the SSH tunnel with tunnel credentials
# Connect to the database
conn = get_db_connection()
conn.autocommit = True
register_vector(conn)
cur = conn.cursor()
# Fetch descriptions from the listings table
# Create the reporters table
# cur.execute("""
# DROP TABLE volumes
# """)
cur.execute("""
CREATE TABLE volumes(
	id text primary key unique,
	volume_number int,
	reporter_slug text,
	data jsonb
);
""")
# conn.commit()
# Fetch the data from the URL

url = f"https://static.case.law/{REPORTER}/VolumesMetadata.json"
response = requests.get(url)
data = response.json()
print(json.dumps(data))
#print(data.get("id"))
# Insert the data into the reporters table
for item in data:
	cur.execute("INSERT INTO volumes (id, volume_number, reporter_slug, data) VALUES (%s, %s, %s, %s)", (item.get("id"), item.get("volume_number"), item.get("reporter_slug"), json.dumps(item)))
conn.commit()
# Close the cursor and connection
cur.close()

server connected


OperationalError: connection failed: connection to server at "127.0.0.1", port 63816 failed: FATAL:  password authentication failed for user "postgres"
connection to server at "127.0.0.1", port 63816 failed: FATAL:  password authentication failed for user "postgres"

## Create reporter Table

In [22]:

# Setting up the SSH tunnel with tunnel credentials
# Connect to the database
conn = get_db_connection()
conn.autocommit = True
register_vector(conn)
cur = conn.cursor()
# Fetch descriptions from the listings table
# Create the reporters table
# cur.execute("""
# DROP TABLE reporters
# """)
# cur.execute("""
# CREATE TABLE reporters(
# 	id text primary key,
# 	data jsonb
# );
# """)
# conn.commit()
# Fetch the data from the URL
url = f"https://static.case.law/{REPORTER}/ReporterMetadata.json"
response = requests.get(url)
data = response.json()
print(json.dumps(data))
#print(data.get("id"))
# Insert the data into the reporters table
cur.execute("INSERT INTO reporters (id, data) VALUES (%s, %s)", (data.get("id"), json.dumps(data)))
conn.commit()
# Close the cursor and connection
cur.close()

server connected
{"id": 477, "full_name": "Washington Appellate Reports", "short_name": "Wash. App.", "start_year": 1969, "end_year": 2017, "jurisdictions": [{"id": 38, "name": "Wash.", "name_long": "Washington"}], "harvard_hollis_id": [], "slug": "wash-app"}


UniqueViolation: duplicate key value violates unique constraint "reporters_pkey"
DETAIL:  Key (id)=(477) already exists.

In [11]:

# Setting up the SSH tunnel with tunnel credentials
# Connect to the database
conn = get_db_connection()
conn.autocommit = True
register_vector(conn)
cur = conn.cursor()
# Fetch descriptions from the listings table
# Create the reporters table
# cur.execute("""
# DROP TABLE cases_metadata
# """)
# cur.execute("""
# CREATE TABLE cases_metadata(
# 	id text primary key unique,
# 	data jsonb
# );
# """)
# conn.commit()

for i in range(NUM_OF_VOLUMES+1, 1, -1):
	# Fetch the data from the URL
	url = f"https://static.case.law/{REPORTER}/{i}/CasesMetadata.json"
	response = requests.get(url)
	data = response.json()
	
	#print(data.get("id"))
	# Insert the data into the reporters table
	
	for item in data:
		cur.execute("SELECT 1 FROM cases_metadata WHERE id = %s", (str(item.get("id")),))
		if cur.fetchone():
			print(url)
			continue
		try:
			cur.execute("INSERT INTO cases_metadata (id, data) VALUES (%s, %s)", (item.get("id"), json.dumps(item)))
			print(json.dumps(data))
			conn.commit()
		except errors.UniqueViolation:
            #print(f"Duplicate key value violates unique constraint: {data.get('id')}")
			conn.rollback()
			
# Close the cursor and connection
cur.close()

server connected


## Create the Cases Table will all the case data in a reporter

In [14]:
from psycopg import errors
# Setting up the SSH tunnel with tunnel credentials
# Connect to the database
conn = get_db_connection()
conn.autocommit = True
register_vector(conn)
cur = conn.cursor()
# Fetch descriptions from the listings table
# Create the reporters table
# cur.execute("""
# DROP TABLE cases
# """)
cur.execute("""
CREATE TABLE IF NOT EXISTS cases(
	id text primary key unique,
	data jsonb
);
""")
conn.commit()

# Fetch the data from the URL
for i in range(NUM_OF_VOLUMES, 0, -1):
    for case in get_links_in_volume(REPORTER,i):
        url = f"https://static.case.law/{REPORTER}/{i}/cases/{case}"
        
        response = requests.get(url)
        data = response.json()
        
        #print(data.get("id"))
        # Insert the data into the reporters table
        #cur.execute("INSERT INTO cases (id, data) VALUES (%s, %s)", (data.get("id"), json.dumps(data)))
        try:
            # Insert the data into the cases table
            print(url)
            cur.execute("INSERT INTO cases (id, data) VALUES (%s, %s)", (data.get("id"), json.dumps(data)))
            
            print(json.dumps(data))
            conn.commit()
        except errors.UniqueViolation:
            #print(f"Duplicate key value violates unique constraint: {data.get('id')}")
            conn.rollback()
        conn.commit()
        # Close the cursor and connection
cur.close()

server connected
https://static.case.law/wash-2d/1/cases/0001-01.json
https://static.case.law/wash-2d/1/cases/0013-01.json
https://static.case.law/wash-2d/1/cases/0017-01.json
https://static.case.law/wash-2d/1/cases/0028-01.json
https://static.case.law/wash-2d/1/cases/0037-01.json
https://static.case.law/wash-2d/1/cases/0041-01.json
https://static.case.law/wash-2d/1/cases/0054-01.json
https://static.case.law/wash-2d/1/cases/0064-01.json
https://static.case.law/wash-2d/1/cases/0079-01.json
https://static.case.law/wash-2d/1/cases/0082-01.json
https://static.case.law/wash-2d/1/cases/0093-01.json
https://static.case.law/wash-2d/1/cases/0102-01.json
https://static.case.law/wash-2d/1/cases/0117-01.json
https://static.case.law/wash-2d/1/cases/0145-01.json
https://static.case.law/wash-2d/1/cases/0165-01.json
https://static.case.law/wash-2d/1/cases/0178-01.json
https://static.case.law/wash-2d/1/cases/0181-01.json
https://static.case.law/wash-2d/1/cases/0196-01.json
https://static.case.law/wash-

In [ ]:

-- Setup Azure OpenAI endpoint
select azure_ai.set_setting('azure_openai.endpoint', 'https://orcaspmopenai.openai.azure.com');
select azure_ai.set_setting('azure_openai.subscription_key', '165c90769b414e829d8dfbd97e84c27b');
select azure_ai.get_setting('azure_openai.endpoint');

In [ ]:
-- Add vector - should take about 5 mins if Azure openAI and Flex are in same region
ALTER TABLE cases
ADD COLUMN description_vector vector(1536) --OPEN AI embeddings are 1536 dimensions
GENERATED ALWAYS AS (
	azure_openai.create_embeddings (
	'text-embedding-3-small', -- example deployment name in Azure OpenAI which CONTAINS text-embedding-ADA-003-small-model
	data#>>'{name}' || LEFT(data#>>'{casebody, opinions, 0}', 8000),
	1536, --dimension
	3600000, --timeouts_ms
	true, --throw_on_error
	2, --max_attempts
	2000 --retry_delay)::vector) STORED; -- TEXT strings concatenated AND sent TO Azure OpenAI


In [ ]:
SELECT description_vector FROM public.cases
where description_vector is not null;


In [14]:
import requests
from bs4 import BeautifulSoup

# URL of the page to scrape
url = 'https://static.case.law/'

# Send a GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all links on the page
links = soup.find_all('a')

# Extract the href attributes
all_links = [link.get('href') for link in links if link.get('href')]

print(all_links[-4])
# Print all the links
# for link in all_links:
#     print(link)


https://static.case.law/yeates/
